<a href="https://colab.research.google.com/github/barrosm/text_mining/blob/master/Aula_20191123_similaridade_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercícios de medidas de similaridade de texto #

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
workdir_path = '/content/drive/My Drive/aula_20191123_leonardo/' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

## Introdução ##

Usaremos [um conjunto de dados de letras de músicas do Kaggle] (https://www.kaggle.com/mousehead/songlyrics) para identificar músicas com letras semelhantes. O conjunto de dados contém artistas, músicas e letras para mais de 55 mil músicas, mas hoje vamos nos concentrar nas músicas de um grupo em particular - os Beatles.

O código a seguir ajudará você a carregar os dados e a se preparar para este exercício.

In [7]:
!pip install nltk

In [0]:
import nltk
import pandas as pd

In [9]:
data = pd.read_csv('songdata.csv')
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


## Desafio  1 ##

* Filtre o conjunto de dados da letra para selecionar apenas músicas dos Beatles.
* Quantas músicas existem no total pelos Beatles?
* Dê uma olhada nas letras da primeira música.

In [10]:
# Veja apenas as músicas dos Beatles
beatles = data[data.artist=='The Beatles']
beatles.head()

,artist,song,link,text
1198,The Beatles,A Shot Of Rhythm And Blues,/b/beatles/a+shot+of+rhythm+blues_20014867.html,"Well, if your hands start a-clappin' \r\nAnd ..."
1199,The Beatles,Across The Universe,/b/beatles/across+the+universe_10026507.html,Words are flowing out like \r\nEndless rain i...
1200,The Beatles,All I've Got To Do,/b/beatles/all+ive+got+to+do_10026646.html,"Whenever I want you around, yeah \r\nAll I go..."
1201,The Beatles,And I Love Her,/b/beatles/and+i+love+her_10026463.html,I give her all my love \r\nThat's all I do \...
1202,The Beatles,And Your Bird Can Sing,/b/beatles/and+your+bird+can+sing_10026364.html,You tell me that you've got everything you wan...


In [11]:
# Verifique o número de músicas dos Beatles no conjunto de dados
data.artist.value_counts().loc[['The Beatles']]

The Beatles    178
Name: artist, dtype: int64

In [12]:
# Veja a letra da primeira música
beatles.iloc[0,3]

"Well, if your hands start a-clappin'  \r\nAnd your fingers start a-poppin'  \r\nAnd your feet start a-movin' around  \r\nAnd if you start to swing and sway  \r\n  \r\nWhen the band starts to play  \r\nA real cool way out sound  \r\nAnd if you get to can't help it and you can't sit down  \r\nYou feel like you gotta move around  \r\n  \r\nYou get a shot of rhythm and blues.  \r\nWith just a little rock and roll on the side  \r\nJust for good measure.  \r\nGet a pair of dancin' shoes  \r\n  \r\nWell, with your lover by your side  \r\nDon't you know you're gonna have a rockin' time, see'mon!  \r\nDon't you worry 'bout a thing  \r\nIf you start to dance and sing  \r\n  \r\nAnd chills come up on you  \r\nAnd if the rhythm finally gets you and the beat gets you too  \r\nWell, here's something for you to do  \r\n  \r\nGet a shot of rhythm and blues  \r\nWith just a little rock and roll on the side  \r\nJust for good measure  \r\nGet a pair of dancin' shoes  \r\n  \r\nWell, with your lover by 

## Desafio 2 ##


Aplique as seguintes etapas de pré-processamento:
* Observe os caracteres '\ n' (nova linha) nas letras. Remova-os usando expressões regulares.
* Remova todas as palavras com números usando expressões regulares.
* Crie uma matriz de termos de documento usando o Count Vectorizer, com cada linha como uma música e cada coluna como uma palavra nas letras. Faça com que o Count Vectorizer remova todas as palavras de parada também.

Nota: O Count Vectorizer remove automaticamente a pontuação e coloca todos os caracteres em minúsculas.

In [13]:
# Remover caracteres da letra
import re

newline = lambda x: re.sub('\n', ' ', x) # remove \n
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x) # remove alphanumeric words

corpus = beatles.text.map(newline).map(alphanumeric)
corpus.head()

1198    Well, if your hands start a-clappin'  \r And y...
1199    Words are flowing out like  \r Endless rain in...
1200    Whenever I want you around, yeah  \r All I got...
1201    I give her all my love  \r That's all I do  \r...
1202    You tell me that you've got everything you wan...
Name: text, dtype: object

In [15]:
# Crie uma matriz de termos de documento usando o Count Vectorizer com as palavras de parada ativadas para inglês
from sklearn.feature_extraction.text import CountVectorizer
          
cv = CountVectorizer(stop_words="english")
X = cv.fit_transform(corpus).toarray()

dt = pd.DataFrame(X, columns=cv.get_feature_names()).set_index(beatles.song)
dt.head()

,aaahhh,aah,abc,aches,aching,acquainted,act,actors,acts,add,admit,adore,advantage,affair,affection,afraid,afternoon,ageing,ago,agree,ah,ahead,ahh,ain,air,alan,albert,allan,alley,allowed,alright,anderen,angel,angry,anna,announcement,answer,anton,anybody,anymore,...,working,works,world,worried,worries,worry,worrying,worse,worst,worth,wouldn,wow,writ,write,writing,wrong,wrote,wuwuwu,ya,yard,yawning,yay,ye,yea,yeag,yeah,year,years,yeh,yellow,yes,yesterday,yoko,young,younger,youre,youu,zealand,zoo,zu
song,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A Shot Of Rhythm And Blues,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
All I've Got To Do,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,2,0,0,0,0,0,0,0,0,0
And I Love Her,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
And Your Bird Can Sing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Desafio  3 ##

* Dê uma olhada na letra da música "Imagine".
* Qual música é a mais parecida com a música "Imagine"?
      * Use a semelhança de cosseno para calcular a semelhança
      * Use o Count Vectorizer para codificar numericamente a letra
* Encontre a música mais parecida usando o TF-IDF Vectorizer.

Compare a música mais semelhante das saídas do Count Vectorizer e do TF-IDF Vectorizer.

In [0]:
# Defina as opções de exibição para que as letras não sejam cortadas
pd.set_option('display.max_colwidth', -1)

In [17]:
#  letra Imagine
beatles[beatles.song=='Imagine'].text

24783    Imagine there's no heaven  \r\nIt's easy if you try  \r\nNo hell below us  \r\nAbove us only sky  \r\nImagine all the people  \r\nLiving for today...  \r\n  \r\nImagine there's no countries  \r\nIt isn't hard to do  \r\nNothing to kill or die for  \r\nAnd no religion too  \r\nImagine all the people  \r\nLiving life in peace...  \r\n  \r\nYou may say I'm a dreamer  \r\nBut I'm not the only one  \r\nI hope someday you'll join us  \r\nAnd the world will be as one  \r\n  \r\nImagine no possessions  \r\nI wonder if you can  \r\nNo need for greed or hunger  \r\nA brotherhood of man  \r\nImagine all the people  \r\nSharing all the world...  \r\n  \r\nYou may say I'm a dreamer  \r\nBut I'm not the only one  \r\nI hope someday you'll join us  \r\nAnd the world will live as one\r\n\r\n
Name: text, dtype: object

In [18]:
# imagine "letra no formato Count Vectorizer"
imagine = list(dt.loc['Imagine'])
imagine[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
# Definir o cálculo de similaridade de cosseno
from numpy import dot
from numpy.linalg import norm

cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

In [20]:
# Calcular todas as semelhanças e classificar pelas mais semelhantes
similarity = [cosine(imagine, song) for song in X]
sorted(list(zip(similarity, beatles.song)), reverse=True)[1:6]

[(0.22757944185316942, "I'll Cry Instead"),
 (0.1738270040532526, 'In My Life'),
 (0.16574838603294895, 'Eleanor Rigby'),
 (0.1490280831849844, 'In Spite Of All The Danger'),
 (0.14592261028699943, 'All My Loving')]

In [21]:
# Crie o vetorizador TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
          
cv_tfidf = TfidfVectorizer(stop_words="english")
X_tfidf = cv_tfidf.fit_transform(corpus).toarray()

dt_tfidf = pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names()).set_index(beatles.song)
dt_tfidf.head()

,aaahhh,aah,abc,aches,aching,acquainted,act,actors,acts,add,admit,adore,advantage,affair,affection,afraid,afternoon,ageing,ago,agree,ah,ahead,ahh,ain,air,alan,albert,allan,alley,allowed,alright,anderen,angel,angry,anna,announcement,answer,anton,anybody,anymore,...,working,works,world,worried,worries,worry,worrying,worse,worst,worth,wouldn,wow,writ,write,writing,wrong,wrote,wuwuwu,ya,yard,yawning,yay,ye,yea,yeag,yeah,year,years,yeh,yellow,yes,yesterday,yoko,young,younger,youre,youu,zealand,zoo,zu
song,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A Shot Of Rhythm And Blues,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.165014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Across The Universe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.342709,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All I've Got To Do,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.29828,0.0,0.0,0.0,0.0,0.120483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
And I Love Her,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
And Your Bird Can Sing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Calcule todas as semelhanças e classifique as mais semelhantes usando os dados do TF-IDF
similarity_tfidf = [cosine(imagine, song) for song in X_tfidf]
sorted(list(zip(similarity_tfidf, beatles.song)), reverse=True)[1:6]

[(0.19266277925834221, "I'll Cry Instead"),
 (0.18004423848592196, "I'll Get You"),
 (0.15513717441169333, 'Eleanor Rigby'),
 (0.15054395512050195, 'In My Life'),
 (0.12306875209548622, 'Love Of The Loved')]

As respostas para o Count Vectorizer e TF-IDF Vectorizer estão bem próximas. Nos dois casos, "Eu vou chorar" é a música mais parecida com "Imagine". Quando você olha as letras, pode ver que as duas estão do lado triste das músicas dos Beatles.

In [23]:
# Imagine lyrics
beatles[beatles.song=='Imagine'].text

24783    Imagine there's no heaven  \r\nIt's easy if you try  \r\nNo hell below us  \r\nAbove us only sky  \r\nImagine all the people  \r\nLiving for today...  \r\n  \r\nImagine there's no countries  \r\nIt isn't hard to do  \r\nNothing to kill or die for  \r\nAnd no religion too  \r\nImagine all the people  \r\nLiving life in peace...  \r\n  \r\nYou may say I'm a dreamer  \r\nBut I'm not the only one  \r\nI hope someday you'll join us  \r\nAnd the world will be as one  \r\n  \r\nImagine no possessions  \r\nI wonder if you can  \r\nNo need for greed or hunger  \r\nA brotherhood of man  \r\nImagine all the people  \r\nSharing all the world...  \r\n  \r\nYou may say I'm a dreamer  \r\nBut I'm not the only one  \r\nI hope someday you'll join us  \r\nAnd the world will live as one\r\n\r\n
Name: text, dtype: object

In [24]:
# I'll Cry Instead lyrics
beatles[beatles.song=='I\'ll Cry Instead'].text

24774    I've got every reason on earth to be mad  \r\n'Cause I just lost the only girl I had  \r\nIf I could get my way  \r\nI'd get myself locked up today  \r\nBut I can't, so I'll cry instead  \r\n  \r\nI've got a chip on my shoulder that's bigger that my feet  \r\nI can't talk to people that I meet  \r\nIf I could see you now  \r\nI'd try to make you sad somehow  \r\nBut I can't, so I'll cry instead  \r\n  \r\nDon't want to cry when there's people there  \r\nI get shy when they start to stare  \r\nI'm gonna hide myself away  \r\nBut I'll come back again someday  \r\n  \r\nAnd when you do you'd better hide all the girls  \r\nI'm gonna break their hearts all round the world  \r\nYes, I'm gonna break them in two  \r\nAnd show you what your lovin' man can do  \r\nUntil then I'll cry instead\r\n\r\n
Name: text, dtype: object

## Desafio 4 ##

Quais são as duas músicas dos Beatles mais parecidas?
    * Usando o Count Vectorizer
    * Usando o TFizer Vectorizer
     
Compare os resultados. Qual vetorizador parece fazer um trabalho melhor?

In [0]:
# Calcular a semelhança de cosseno entre todas as combinações de documentos

In [26]:
# Listar todas as combinações de músicas
from itertools import combinations

pairs = list(combinations(beatles.song.index, 2)) # all song index combos
pairs_0 = list(combinations(range(len(beatles)), 2)) # all index combos starting with (0,1)
song_pairs = [(beatles.song[a_index], beatles.song[b_index]) for (a_index, b_index) in pairs]
song_pairs[:5]

[('A Shot Of Rhythm And Blues', 'Across The Universe'),
 ('A Shot Of Rhythm And Blues', "All I've Got To Do"),
 ('A Shot Of Rhythm And Blues', 'And I Love Her'),
 ('A Shot Of Rhythm And Blues', 'And Your Bird Can Sing'),
 ('A Shot Of Rhythm And Blues', 'Another Girl')]

In [27]:
# Calcular a semelhança de cosseno da matriz termo-documento do Count Vectorizer
results = [cosine(X[a_index], X[b_index]) for (a_index, b_index) in pairs_0]
sorted(zip(results, song_pairs), reverse=True)[:5]

[(0.8548876342047523, ('All You Need Is Love', 'Love Me Do')),
 (0.8359173188630292, ('If I Needed Someone', 'If I Needed Someone To Love')),
 (0.8058125835571367, ('And I Love Her', 'Love Me Do')),
 (0.7930110315846619, ('If I Fell', 'Love Me Do')),
 (0.7478391382739419, ('And I Love Her', 'All You Need Is Love'))]

In [28]:
# Calcular a semelhança de cosseno da matriz termo-documento do TF-IDF Vectorizer
results = [cosine(X_tfidf[a_index], X_tfidf[b_index]) for (a_index, b_index) in pairs_0]
sorted(zip(results, song_pairs), reverse=True)[:5]

[(0.8717047783261522, ('If I Needed Someone', 'If I Needed Someone To Love')),
 (0.6599353568343105, ('All You Need Is Love', 'Love Me Do')),
 (0.6502842517883833, ('Have A Banana!', 'Crinsk Dee Night')),
 (0.6232797327325461, ("Don't Let Me Down", 'Let It Be')),
 (0.5275364336322862, ('If I Fell', 'Love Me Do'))]


Com o Count Vectorizer, todas as músicas principais contêm o termo "love", com "All You Need Is Love" e "Love Me Do" sendo as mais semelhantes. Isso ocorre porque os Beatles tendem a usar muito o termo "amor" em suas músicas.

Com o TF-IDF Vectorizer, as principais músicas semelhantes são um pouco mais interessantes. Parece que ele realmente pegou uma música duplicada no conjunto de dados das letras com "If I Needed Someone" e "If I Needed Someone To Love". Além disso, "Tenha uma banana!" e "Crinsk Dee Night" são conversas com o apresentador Brian Matthew.

In [29]:
# All You Need Is Love lyrics
beatles[beatles.song=='All You Need Is Love'].text

24695    Love, love, love, love, love, love, love, love, love.  \r\n  \r\nThere's nothing you can do that can't be done.  \r\nNothing you can sing that can't be sung.  \r\nNothing you can say, but you can learn  \r\nHow to play the game  \r\nIt's easy.  \r\nNothing you can make that can't be made.  \r\nNo one you can save that can't be saved.  \r\nNothing you can do, but you can learn  \r\nHow to be you in time  \r\nIt's easy.  \r\n  \r\nAll you need is love, all you need is love,  \r\nAll you need is love, love. Love is all you need.  \r\nLove, love, love, love, love, love, love, love, love.  \r\nAll you need is love, all you need is love,  \r\nAll you need is love, love. Love is all you need.  \r\n  \r\nThere's nothing you can know that isn't known.  \r\nNothing you can see that isn't shown.  \r\nThere's nowhere you can be that isn't where  \r\nYou're meant to be  \r\nIt's easy.  \r\n  \r\nAll you need is love, all you need is love,  \r\nAll you need is love, love. Love is all you ne

In [30]:
# Love Me Do lyrics
beatles[beatles.song=='Love Me Do'].text

24805    Love, love me do.  \r\nYou know I love you,  \r\nI'll always be true,  \r\nSo please, love me do.  \r\nWhoa, love me do.  \r\n  \r\nLove, love me do.  \r\nYou know I love you,  \r\nI'll always be true,  \r\nSo please, love me do.  \r\nWhoa, love me do.  \r\n  \r\nSomeone to love,  \r\nSomebody new.  \r\nSomeone to love,  \r\nSomeone like you.  \r\n  \r\nLove, love me do.  \r\nYou know I love you,  \r\nI'll always be true,  \r\nSo please, love me do.  \r\nWhoa, love me do.  \r\n  \r\nLove, love me do.  \r\nYou know I love you,  \r\nI'll always be true,  \r\nSo please, love me do.  \r\nWhoa, love me do.  \r\nYeah, love me do.  \r\nWhoa, oh, love me do.\r\n\r\n
Name: text, dtype: object

In [31]:
# If I Needed Someone lyrics
beatles[beatles.song=='If I Needed Someone'].text

24768    If I needed someone to love  \r\nYou're the one that I'd be thinking of  \r\nIf I needed someone  \r\n  \r\nIf I had some more time to spend  \r\nThen I guess I'd be with you my friend  \r\nIf I needed someone  \r\nHad you come some other day  \r\nThen it might not have been like this  \r\nBut you see now I'm too much in love  \r\n  \r\nCarve your number on my wall  \r\nAnd maybe you will get a call from me  \r\nIf I needed someone  \r\nAh, ah, ah, ah  \r\n  \r\nIf I had some more time to spend  \r\nThen I guess I'd be with you my friend  \r\nIf I needed someone  \r\nHad you come some other day  \r\nThen it might not have been like this  \r\nBut you see now I'm too much in love  \r\n  \r\nCarve your number on my wall  \r\nAnd maybe you will get a call from me  \r\nIf I needed someone  \r\nAh, ah\r\n\r\n
Name: text, dtype: object

In [32]:
# If I Needed Someone To Love lyrics
beatles[beatles.song=='If I Needed Someone To Love'].text

24769    If I needed someone to love  \r\nYoure the woman I'd be thinking of  \r\nIf I needed someone  \r\n  \r\nIf I had some more time to spend  \r\nThen I guess I'd be with you my friend  \r\nIf I needed someone  \r\n  \r\nHad you come some other day  \r\nThen it might not have been like this  \r\nBut you see now I'm too much in love  \r\n  \r\nCarve your number on my wall  \r\nAnd maybe you will get a call from me  \r\nIf I needed someone  \r\n  \r\nIf I had some more time to spend  \r\nThen I guess I'd be with you my friend  \r\nIf I needed someone  \r\n  \r\nHad you come some other day  \r\nThen it might not have been like this  \r\nBut you see now I'm too much in love  \r\n  \r\nCarve your number on my wall  \r\nAnd maybe you will get a call from me  \r\nIf I needed someone  \r\nIf I needed someone to love  \r\nYoure the woman I'd be thinking of  \r\nIf I needed someone  \r\n  \r\nIf I had some more time to spend  \r\nThen I guess I'd be with you my friend  \r\nIf I needed someo

In [33]:
# Have a Banana! lyrics
beatles[beatles.song=='Have A Banana!'].text

1221    [Speech]  \r\n  \r\nBrian Matthew: Is that it? Is that the end?  \r\nPaul: Yeah, yeah, that's it.  \r\nJohn: Fade, fade!  \r\nBrian: Good track. Oh, well, we'll stop there, stop there, stop there.  \r\nJohn: What an end!  \r\nBrian: Quiet! All right, George.  \r\nJohn: Fade!  \r\nBrian: Hold it!  \r\nGeorge: Oh, thank you.  \r\nJohn: Fade, you silly.  \r\nBrian: Well, we did. We did that. Oh, no! No! We've done that bit!  \r\nJohn: The train comes in now.  \r\nBrian: We did that.  \r\nJohn: Yeah.  \r\nBrian: To pove we weren't playing the record, then, you see. 'Cause,\r\notherwise, there's no point in you being here, is there? Ha, ha, ha!  \r\nJohn: Yeah, we did that, 'cause it sounds just like it, don't it?  \r\nBrian: Pretty cool lot of fellows, aren't you? Here, Ringo, have a banana,\r\ncatch!\r\n\r\n
Name: text, dtype: object

In [34]:
# Crinsk Dee Night lyrics
beatles[beatles.song=='Crinsk Dee Night'].text

24713    [Speech]  \r\n  \r\nBrian Matthew: The next few minutes, we're in the lap of the gods and the\r\nhands of the Beatles. In my young days, when I was a lad, they used to have\r\nactors in films and now that they--  \r\nPaul: Yes?  \r\nJohn: Hey! Listen!  \r\nPaul: It's all changed, now, Brian. They're not doing that, no actors.  \r\nJohn: It's all changed, now.  \r\nBrian: But this is what I wonder. In those days, the actors used to say their\r\nbest bits were left on the cutting room floor. Did you find that?  \r\nJohn: No, no, no, those were the good bits in the film. You should have seen\r\nthe rest.  \r\nBrian: Yes?  \r\nJohn: Rubbish!  \r\nBrian: Was it, really?  \r\nJohn: Even worse, yes.  \r\nBrian: Who was worst?  \r\nJohn: Oh, Paul.  \r\nBrian: I see.  \r\nPaul: I think John was about the worst.  \r\nJohn: No, it was you.  \r\nPaul: Oh, Ringo was very good. He was. He's a good lad.  \r\nBrian: He was. They're saying he's a new Charlie Chaplin. Do you think that's\r\nrig

In [0]:
#### Minha análise

In [37]:
# Músicas de Eagles
eagles = data[data.artist=='Eagles']
eagles.head()

,artist,song,link,text
4629,Eagles,Chug All Night,/e/eagles/chug+all+night_20044427.html,"You scare me a bit, \r\nBut that's all right \r\nYou know when I want you \r\nMost every night \r\n \r\nAnd I've been meaning to tell \r\nYou baby that it makes no sense \r\nStill I'm finally convinced \r\nYeah yeah \r\n \r\nI believe we could chug all night \r\nI believe we could hug all night \r\nThe band is loose and the groove \r\nIs right you're so much woman I \r\n \r\nBelieve we could chug all night \r\nOn the day that I die, well \r\nI just might scream \r\nIf I'm alive in the morning \r\n \r\nI'll be alive in a dream \r\nYou better listen to me baby \r\nCause you know that I'm hung on you \r\nTill I'm blind and black and blue \r\n \r\nNo one else will do \r\nI do believe we chug all night \r\nI believe we could hug all night \r\nThe band is loose and the groove is right \r\n \r\nYou're so much woman I believe \r\nWe could chug all night \r\nNo woman ever do what you do \r\nHigh on a pleasure wheel \r\n \r\nNo devil ever cast a voodoo \r\nSo long and dark and real \r\nWe're gonna do a little chugging \r\nWe're gonna do a little hugging \r\n \r\nThe band is loose and the groove is right \r\nI'm wired for sound \r\nAre you wired for light? \r\nAnd you're so much woman, \r\n \r\nI believe we could chug all night \r\nI said a yeah yeah yeah \r\nYeah yeah \r\nWoo! it's only midnight baby \r\nCome on and do it to me\r\n\r\n"
4630,Eagles,Doolin-Dalton,/e/eagles/doolin+dalton_20044420.html,"They were doolin', doolin'-dalton \r\nHigh or low, it was the same \r\nEasy money, and faithless women \r\nRed eye whiskey for the pain. \r\n \r\nGo down little dalton, it must be God's will \r\nTwo brothers lyin' dead in Coffeeville. \r\nTwo voices call to you from where they stood, \r\n""Lay down your law books now, they're no damn good."" \r\n \r\nBetter keep on movin', doolin'-dalton \r\nTill your shadow sets you free. \r\nAnd if you're fast, and if you're lucky, \r\nYou will never see that hangin' tree. \r\n \r\nWell the towns lay out across the dusty plain \r\nLike graveyards filled with tombstones waitin' for the names \r\nAnd a man could use his back or use his brains \r\nBut some just went stir-crazy Lord cause nothin' ever changed \r\n \r\nTill they're doolin' Neville Dalton \r\nHe was workin' cheap, just buyin' time \r\nAnd then he laughed and said I'm goin' \r\nAnd so he left that peaceful life behind\r\n\r\n"
4631,Eagles,Falling Down The Stairs,/e/eagles/falling+down+the+stairs_20550829.html,"I don't know what you say \r\nEvery single holy day, \r\nNever in my life I see \r\nHow can it be, what's it for me? \r\n \r\nI know that you know that we know that you know \r\nEvery what's his name \r\nI know that you know that we know that you know \r\nWhat, how can it be? \r\nTaking a tumble you know a tumble that you take a tumble \r\n(Its not like this) \r\nCos I'm falling down the stairs, Help me cos I've got no grip, oh help me\r\nplease \r\n(And I fall) \r\n \r\nShe left me, I can't see \r\nI'm blind \r\nHelp me I'm so strong \r\nAnd then I lose my mind \r\nI want to be there but you cannot see me, with I know what's in your head \r\n \r\nI know that you know that we know that you know \r\nEvery what's his name \r\nI know that you know that we know that you know \r\nWhat, how can it be? \r\nTaking a tumble you know a tumble that you take a tumble \r\n(Its not like this) \r\nCos I'm falling down the stairs, Help me cos I've got no grip \r\n(And I fall)\r\n\r\n"
4632,Eagles,I Wish You Peace,/e/eagles/i+wish+you+peace_20044528.html,"I wish you peace when the cold winds blow \r\nWarmed by the fire's glow \r\nI wish you comfort in the, the lonely time \r\nAnd arms to hold you when you ache inside \r\n \r\nI wish you hope when things are going bad \r\nKind words when times are sad \r\nI wish you shelter from the, the raging wind \r\nCooling waters at the fever's end \r\n \r\nI wish you peace when times are hard \r\nThe light to guide you through th

In [38]:
data.artist.value_counts().loc[['Eagles']]

Eagles    41
Name: artist, dtype: int64

In [39]:
data.artist.value_counts().loc[['America']]

America    184
Name: artist, dtype: int64

In [42]:
data.artist.value_counts().loc[['Fleetwood Mac']]

Fleetwood Mac    180
Name: artist, dtype: int64

In [45]:
eagles.song

4629     Chug All Night                  
4630     Doolin-Dalton                   
4631     Falling Down The Stairs         
4632     I Wish You Peace                
4633     It's Your World Now             
4634     James Dean                      
4635     Life's Been Good                
4636     My Man                          
4637     Outlaw Man                      
4638     Peaceful Easy Feeling           
4639     Please Come Home For Christmas  
4640     Pretty Maids All In A Row       
4641     Seven Bridges Road              
4642     Take The Devil                  
4643     The Girl From Yesterday         
4644     Try And Love Again              
4645     Tryin'                          
30712    All By Myself                   
30713    All Night Long                  
30714    Already Gone                    
30715    Bitter Creek                    
30716    Earlybird                       
30717    Guilty Of The Crime             
30718    Hotel California (Album V

In [52]:
# Mudando o corpus para músicas dos Eagles

corpus = eagles.text.map(newline).map(alphanumeric)

cv = CountVectorizer(stop_words="english")

X = cv.fit_transform(corpus).toarray()


dt = pd.DataFrame(X, columns=cv.get_feature_names()).set_index(eagles.song)
dt.head()

,abel,accountants,ache,add,address,aflight,afraid,ago,ah,ahead,ain,air,al,alibies,alive,allow,allowed,alright,angels,answering,antone,anymore,apart,appear,apple,arizona,arms,arriving,asking,attacked,auto,aw,awaken,away,awright,baby,bad,band,basketball,beast,...,wheel,wheels,whine,whiskey,whisperin,whistle,wild,win,wind,window,winds,wine,winslow,wired,wish,wishing,woman,women,won,wonder,wonderin,woo,words,work,workin,working,world,worry,wouldn,write,wrong,wrote,ya,yeah,year,yearning,years,yes,yesterday,young
song,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Chug All Night,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0
Doolin-Dalton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Falling Down The Stairs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
I Wish You Peace,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,12,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
It's Your World Now,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
hotel_cal= list(dt.loc['Hotel California (Album Version)'])
hotel_cal[0:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]

In [54]:
# Calcular todas as semelhanças e classificar pelas mais semelhantes com Hotel California
similarity_hotel_cal = [cosine(hotel_cal, song) for song in X]
sorted(list(zip(similarity, eagles.song)), reverse=True)[1:10]

[(0.1344743464189073, 'Doolin-Dalton'),
 (0.12472767145898403, "Life's Been Good"),
 (0.12154321660528142, 'How Long'),
 (0.11542497475994799, 'Music'),
 (0.11378972092658471, "It's Your World Now"),
 (0.11155467020454342, 'My Man'),
 (0.09362448629677092, 'Too Many Hands'),
 (0.09013207588042399, 'Outlaw Man'),
 (0.07915594835766294, 'Nightingale')]